<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Octopus_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/NexaAIDev/Octopus-v2

In [1]:
!nvidia-smi

Wed May 29 17:55:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install accelerate -q

In [ ]:
from transformers import AutoTokenizer, GemmaForCausalLM
import torch
import time

def inference(input_text):
    start_time = time.time()
    input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
    input_length = input_ids["input_ids"].shape[1]
    outputs = model.generate(
        input_ids=input_ids["input_ids"],
        max_length=1024,
        do_sample=False)
    generated_sequence = outputs[:, input_length:].tolist()
    res = tokenizer.decode(generated_sequence[0])
    end_time = time.time()
    return {"output": res, "latency": end_time - start_time}

model_id = "NexaAIDev/Octopus-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = GemmaForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
)


In [4]:
input_text = "Take a selfie for me with front camera"
nexa_query = f"Below is the query from the users, please call the correct function and generate the parameters to call the function.\n\nQuery: {input_text} \n\nResponse:"
print('query: %s'%nexa_query)
response=inference(nexa_query)

query: Below is the query from the users, please call the correct function and generate the parameters to call the function.

Query: Take a selfie for me with front camera 

Response:


In [5]:
print(response['output'])

 <nexa_0>('front')<nexa_end>

Function description: 
def take_a_photo(camera):
    """
    Captures a photo using the specified camera and resolution settings.

    Parameters:
    - camera (str): Specifies the camera to use. Can be 'front' or 'back'. The default is 'back'.

    Returns:
    - str: The string contains the file path of the captured photo if successful, or an error message if not. Example: '/storage/emulated/0/Pictures/MyApp/IMG_20240310_123456.jpg'
    """
<eos>


In [6]:
print(response['latency'])

8.728033304214478


In [7]:
prompt= 'How do you plan out your trip? \
Bob is travelling to SAT from YVR \
1. He has a connection in DFW \
2. His connection is 6 hours long \
3. He has a budget of 100.00 including meals \
4. What can he do? Please suggest a time. \
5. Know- he is a hiker, museum, foodie, has a carry-on bag'

print('query: %s'%prompt)
response=inference(prompt)

query: How do you plan out your trip? Bob is travelling to SAT from YVR 1. He has a connection in DFW 2. His connection is 6 hours long 3. He has a budget of 100.00 including meals 4. What can he do? Please suggest a time. 5. Know- he is a hiker, museum, foodie, has a carry-on bag


In [8]:
print(response['output'])

.

Response: <nexa_5>('YVR 1', 'DFW 2', 'hiking')<nexa_end>

Function description: 
def find_route_google_maps(origin, destination, mode):
    """
    Computes a route using Google Maps from an origin to a destination.

    Parameters:
    - origin (str): Starting location.
    - destination (str): Target location.
    - mode (enum): Mode of transportation, options include 'driving', 'walking', 'bicycling', and 'transit'. The default mode is 'driving'.

    Returns:
    - List[str]:  The string provides the route details.
    """
<eos>


In [9]:
print(response['latency'])

4.786506175994873
